In [ ]:
import numpy as np
import pandas as pd
import polarDensity_helper as pc
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
plt.rcParams['axes.grid'] = False 

# Important User Settings:

In [ ]:
# root = Path("/home/ems363/Projects/ELIC_PE_PG/trajectory_version")
# chains_groups = ["POPE", "POPG"]
#root = Path("/home/ems363/Projects/ELIC_PE_PG/aggregated/")
root = Path("/home/ems363/Projects/ELIC_PE_PG/aggregated/liam/")
#root = Path("/home/ems363/Projects/ELIC_PE_PG/liam/trajectory_version/")
chains_groups = ["POPG", "POPE"]
lipids = chains_groups

In [ ]:
enrich = True

# get files to use

file_list = []
for lip in lipids:
    toadd = list(root.glob(f"{lip}.dat*avg.dat") )
    file_list = np.append(file_list,toadd)

leaflets = ['low', 'upp']

In [ ]:
enrichments = pd.DataFrame(index=chains_groups, columns=leaflets)
counts = pd.DataFrame(index=chains_groups, columns=leaflets)

idx = 0
for fl in file_list:
    if idx == 0:
        rad, dr, dth, theta, radius, frames, Ntheta = pc.Coord_Get(fl)

    filename = fl.name

    tmp_chain = filename.split('.')[0]
    tmp_nm = filename.split('.')[2]

    # This is a hack. The above part does not have a "flexible"
    # method to consider sim type (a, b ...)
    idx+=1
    toadd = np.loadtxt(fl, skiprows=1)
    toadd = toadd[:,3:-1]
    counts.at[tmp_chain,tmp_nm] = toadd
    enrichments.at[tmp_chain,tmp_nm] = pc._analysis_call_(fl, radius, dr, dth, frames, enrich=enrich)

# Optional helix locations
try:
    helices_lwr = np.loadtxt(root.joinpath("Protein_coords_lwr.dat"))
    helices_upr = np.loadtxt(root.joinpath("Protein_coords_upr.dat"))
except FileNotFoundError:
    helices_lwr = None
    helices_upr = None
    print("Protein coordinates not found")

In [ ]:
thetas = np.unique(theta.flatten())

In [ ]:
bins = {1:(range(4,30,6), np.repeat(27.5,5)), 
        2:(range(5,30,6), np.repeat(27.5,5)),
        3:(range(4,30,6), np.repeat(32.5,5)),
        4:(range(5,30,6), np.repeat(32.5,5)),
        5:(range(4,30,6), np.repeat(22.5,5))}

In [ ]:
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('bwr_r', 256)
newcolors = depleted(np.linspace(0, 1, 256))
newcolors[128:] = enriched(np.linspace(0.5,1,128))
my_cmap = ListedColormap(newcolors)

In [ ]:
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('PiYG', 256)
enriched = plt.cm.get_cmap('bwr_r', 256)
newcolors = np.concatenate([depleted(np.linspace(0.4, 0.5, 128)), enriched(np.linspace(0.5,1,128))])
my_cmap = ListedColormap(newcolors)

In [ ]:

enrichments.at['POPE', 'low'][0:1,:]=0
fig, axes = polar_plot(enrichments, 
                       theta, 
                       radius, 
                       chains_groups, 
                       helices_lwr, 
                       helices_upr, 
                       colorbychain=False, 
                       vmin=0.75, 
                       vmax=1.5, 
                       vmid=1,
                       figheight=8,
                       figwidth=8,
                       cmap=my_cmap)
fig.tight_layout()
plt.savefig(root.joinpath("ELIC_enrichments.pdf"))
plt.show()